In [ ]:
#Task 4

The process for the stream frame-work is as follows :

1.	An activity is first defined, the activity basically has the following
a.	User Id 
b.	Verb : What is the action : Tweeting , liking etc.
c.	Content : The actual tweet/post
d.	Any extra information regarding the activity

2.	 Two feeds are then created – One the user’s personal feed and the Redis feed where all the followed tweets are stored.

3.	Finally the activity is then sent to all the followers , the process is called FanOut using the manager class.
Now Stream-Frame work uses cerly to do the fanout operations. Cerly has two queues, one which is the high priority and the 2nd is the low priority. Active users are assigned the high priority.


In [ ]:
def create_activity(pin):
    from stream_framework.activity import Activity
    activity=Activity(Pin.user_id,PinVerb,pin.id,pin.influncer_id,
                     time=make_naive(pin.created_at,pytz.utc),
                     extra_contexts=dict(item_id=pin.item_id))
    return activity

In [ ]:
from stream_framework.feeds.redis import RedisFeed
class PinFeed(RedisFeed):
    key_format='feed:normal:%(user_id)s'
class PinFeed(RedisFeed):
    key_format='feed:user:%(user_id)s'    

In [ ]:
from stream_framework.feed_managers.base import Manager
class PinManager(Manager):
    feed_classes=dict(normal=PinFeed,)
    user_feed_class=UserPinFeed
    
    def add_pin(self,pin):
        activity=pin.create_activity()
        self.add_user_activity(pin.user_id,activity)
        
    def get_user_follower_ids(self,user_id):
        ids=Follow.objects.filter(target=user_id).value_list('user_id',flat=True)
        return {FanoutPriority.HIGH:ids}
manager=PinManager()

The Redis store is nothing but a message box,i.e. a message storage for every user. Whenever required each user gets all its messages from Redis.

Source : http://thenewstack.io/building-scalable-news-feed-applications-using-redis-and-cassandra/
